# Model Experiments

In [6]:
%matplotlib inline
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
from torchvision import datasets

## Process data

In [ ]:
import torchvision.transforms as transforms
from torchvision.datasets import FashionMNIST

DATA_DIR = "./data"
transformation = transforms.Compose([transforms.ToTensor()])

# mnist_real_train = datasets.FashionMNIST(DATA_DIR, train=True, download=False)
# mnist_real_test = datasets.FashionMNIST(DATA_DIR, train=False, download=False)
# mnist_real_test[0][0]

mnist_real_train = datasets.FashionMNIST(DATA_DIR, train=True, download=False, transform=transformation)
mnist_real_test = datasets.FashionMNIST(DATA_DIR, train=False, download=False, transform=transformation)

mnist_real_train[0][0]


torch.Tensor